탐색적 자료 분석에서 벗어나 웹 서버와 웹 서비스를 자세히 살펴본다. 이들은 어느 정도 함수의 연쇄라 할 수 있다. 웹 콘텐츠를 제공하는 문제에 대해 몇 가지 함수형 설계를 적용할 수 있다. 표현적 상태 전송(Representational State Transful, REST)에 접근하는 방법을 샆벼노는 것이다. 함수형 설계 패턴을 사용해 REST적인 웹 서비스를 구축하고 싶다.

대부분의 프레임워크에 적용할 수 있는 몇 가지 원칙을 보여줄 것이다. 함수형 디자인 패턴을 사용해 웹 콘텐츠를 제공할 수 있어야 한다. 이를 통해 함수형 설계의 장점을 살리는 웹 기반 애플리케이션을 만들 수 있다. 

예를 들어, 극단적으로 큰 데이터 집합을 살펴보거나 극단적으로 복잡한 데이터 집합을 보는 경우, 부분 집합 생성이나 검색을 지원하는 웹 서비스가 있으면 좋을 것이다. 데이터의 부분 집합을 여러 가지 형식으로 다운로드할 수 있는 웹 사이트가 필요할 수도 있다. 이렇게 더 복잡한 요구사항을 지원하는 REST적인 웹 서비스를 만들기 위해 함수형 설계가 필요할 수도 있다. 

대부분의 복잡한 웹 애플리케이션에는 사이트 사용을 쉽게 하기 위해 상태가 있는 세션이 존재한다. 세션 정보는 HTML 양식을 통해 제공한 데이터를 통해 갱신되건, 데이터베이스에서 불러오거나, 클라이언트와 서버의 이전 상호작용에 대한 캐시에서 되살릴 수 있다. 전체 상호작용에는 상태 변경이 포함되지만, 애플리케이션 프로그래밍은 상당 부분이 함수적이다. 애플리케이션의 기능 중 상당 부분은 요청 데이터나 캐시 데이터, 또는 데이터베이스 객체의 사용 여부에 대해 그리 엄격하지 않을 수 있다.

구체적인 웹 프레임워크를 자세히 알아야 할 필요를 피하기 위해 웹 서버 게이트웨이 인터페이스 디자인 패턴에 집중할 것이다. WSGI를 사용하면 간단한 웹 서버를 구현할 수 있다. 

HTTP 프로토콜을 살펴보는 것부터 시작할 것이다. 그 후 아파치 httpd 등을 사용해 프로토콜을 구현하는 방법과 mod_wsgi가 기본 서버를 확장하는 합리적인 방법인 이유를 살펴본다. 이러한 배경을 바탕으로 WSGI의 함수적인 특성을 살펴보고, 함수형 설계를 활용해 복잡한 웹 검색과 데이터 입수 도구를 구현할 수 있는지 살펴본다.

### HTTP 요청 - 응답 모델

HTTP 프로코톨의 핵심은 이상적인 경우, 상태가 없다. 사용자 에이전트 또는 클라이언트는 프로토콜에 대해 함수적인 관점을 가질 수 있따. http.client나 urllib 라이브러리를 사용하면 클라이언트를 만들 수 있다. HTTP 사용자 에이전트는 기본적으로 다음과 비슷한 기능을 실행한다.

In [ ]:
import urllib.request
with urllib.request.urlopen("http://slott-softwarearchitect.blogspot.com") as response:
    print(response.read())

wgert이나 curl과 같은 프로그램은 이러한 동작을 명령 행에서 수행한다. 이떄 인자에서 접속할 URL을 가져온다. 브라우저는 사용자의 마우스 이동과 클릭에 대한 응답으로 이러한 동작을 수행한다. 이떄 사용자의 행동, 구체적으로 말하자면 링크가 걸린 텍스트나 이미지를 클릭하는 행동으로부터 URL을 가져온다. 

하지만 서로 연결된 서버와 클라이언트 간의 상호 작용을 실용적으로 만들기 위해 상태가 있는 몇 가지 세부 구현이 생겼다. 일부 HTTP 상태 코드는 사용자 에이전트 쪽에서 추가 동작을 실행해야 함을 표현한다.

300번대 상태 코드는 요청 받은 자원이 다른 데로 이동했다는 것을 나타낸다. 그러한 응답을 받으면 사용자 에이전트는 Location 헤더의 정보에 따라 새로운 위치로 요청을 보내야 한다. 401 상태 코드는 인증이 필요하다는 뜻이다. 그러한 경우, 사용자 에이전트는 서버 접근을 위해 필요한 인증 정보가 들어간 헤더를 포함시켜 다시 요청을 보내야 한다. urllib 라이브러리에는 이러한 식의 상태가 존재하는 추가적인 부분의 구현도 들어 있다. http.client는 자동으로 300번대 위치 재지정 코드를 처리하지 않는다.

사용자 에이전트가 300번대나 401 코드를 처리하는 기법은 그렇게 샅애와 깊은 연관이 있지는 않으므로 간단한 재귀를 사용할 수 있다. 상태 코드와 위치 재지정이 아니라면, 그 경우가 기본적인 경우이며, 처리 함수는 최종 결과를 얻은 것이다. 위치 재지정이 필요하면 대상 주소를 가지고 처리 함수를 재귀호출할 수 있다. 

프로토콜의 반대쪽 끝을 보자. 정적인 콘텐츠를 제공하는 서버는 상태가 없어도 된다. HTTP 프로토콜에는 TCP/IP 소켓과 그 소켓을 활용하는 더 고수준의 HTTP구조, 이렇게 두 가지 계층이 존재한다. 저수준 처리는 socketserver 라이브러리가 담당하며, 파이썬의 http.server 라이브러리를 고수준 구현을 제공하는 여러 라이브러리 중 하나다.

http.server 라이브러리를 다음과 같이 사용할 수 있다.

In [ ]:
from http.server import HTTPServer, SimpleHTTPRequestHandler
running = True
httpd = HTTPServer(('localhost', 8099), SimpleHTTPRequestHandler)
while running:
    httpd.handle_request()
httpd.shutdown()

서버 객체를 만들고 httpd 변수에 그 객체를 대입했다. 외부에서 들어오는 요청을 기다릴 주소와 포트 번호를 제공했다. 해당 포트로 연결 요청이 들어오면 TCP/IP 프로토콜은 별도의 포트를 하나 할탕해 연결을 맺는다. HTTP 프로토콜은 새로 할당된 포트로부터 요청 데이터를 읽고 처리할 핸들러 handler의 인스턴스를 생성한다.

이 예제에서는 SimpleHTTPRequestHandler를 각 요청을 처리할 핸들러를 인스턴스화할 클래스로 지정했다. 이 클래스는 클라이언트에게 응답 헤더를 보낸 후에 다시 응답 본문을 보내는 최소한의 인터페이스를 구현해야 한다. 이 클래스는 로컬 디렉터리에 있는 파일을 